## Import libraries

In [1]:
import numpy as np
from pdf2image import convert_from_path
from pytesseract import image_to_string
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../utils")
import utils as uti
import pandas as pd
import scipy
import os
from tqdm import tqdm

## Create dataset from GA state raw data and save it as csv file

In [ ]:

DATASET = "GA"
directory_path = "../data/raw/"+DATASET
print(directory_path)
df = pd.DataFrame(columns=['Descriptions'])

peaks_heights = list(np.arange(0.1,0.4,0.005))
files = [file for file in os.listdir(directory_path) if file.endswith('.pdf')]
for file in tqdm(files, desc="Processing PDF files"):
    pages = convert_from_path(directory_path+"/"+file, 200)
    text = ""
    for page in pages:
        page_array = np.array(page)
        img = uti.rgb_to_gray(page_array)
        mean_img = (256-np.mean(img, axis=1))/256
        opt_peaks = []
        predist = int(0.01*len(mean_img))
        for i in peaks_heights:
            lines, _= scipy.signal.find_peaks(mean_img, height = i, distance=predist)
            if len(lines)>=4:
                dist=(lines-np.roll(lines,1))[1:]
                opt_peaks.append(np.std(dist))
            else:
                opt_peaks.append(np.inf)

        best_height = peaks_heights[opt_peaks.index(min(opt_peaks))]
        descr_lines, _ = scipy.signal.find_peaks(mean_img, height=best_height, distance=predist)
        if len(descr_lines)>1:
            tol =  min((descr_lines - np.roll(descr_lines, 1))[1:])
        else:
            tol = 50
        trim_img = page_array[max(descr_lines[0]-tol, 0): min(descr_lines[-1]+tol, len(mean_img)-1)]
        text += image_to_string(trim_img, lang='eng')
    df = pd.concat([df, pd.DataFrame({'Descriptions': [text]})], ignore_index=True)

df.to_csv("../data/csv_files/"+ DATASET+'_disciplinary_actions.csv', index=False)